<a href="https://colab.research.google.com/github/VictorHugoMartins/israel_x_palestine_data_analysis/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpeza de Comentários

Redução do vocabulário dos comentários coletados. Vídeos e Canais não passam por esse processo por não serem o foco da pesquisa e servirem como bases auxiliares para o percurso principal.

In [1]:
from config import result_index

execution = 'local'
# execution = 'cloud'

base_path = 'data' if execution == 'local' else 'cloud'

In [2]:
import pandas as pd

unified_df = pd.read_csv(f"processed_data_{result_index}/cleaned_videos.csv")

In [3]:
unified_df.sample(5)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,video_id,title,description,channel_id,published_at,category_id,tags,...,concurrent_viewers,active_live_chat_id,recording_date,topicCategories,processing_status,parts_total,parts_processed,time_left_ms,processing_failure_reason,text
395,395,395.0,434.0,0aSUMX1KbgU,MULHER DIZ QUE VAI PARA O BAR COM AS AMIGAS E ...,seja bem vindo a mais um vídeo do canal Não se...,UCUBeVY6Kn7ulBmUGynJqISw,2025-05-04 03:22:46,24,"['redpill', 'msol', 'miqueinha', 'relacionamen...",...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Humour'],NaN,0,0,0,NaN,mulher diz que vai para bar com amigas e homem...
391,877,NaN,NaN,3kGp6UEEgNY,PARE de ignorar esses sinais de que ele só ama...,GRUPO TELEGRAM👇\nhttps://bit.ly/CanalRedpillfe...,UCO9FRrBUwGdYopkMbGGKbpg,2025-05-04 13:00:06,27,"['como fazer ele sentir sua falta', 'amor e au...",...,0,NaN,2025-05-04T00:00:00Z,[],NaN,0,0,0,NaN,pare de ignorar esses sinai de que ele só ama ...
353,353,353.0,390.0,uyh7hDSfM84,A luta silenciosa dos homens #redpill #masculi...,💥 QUER VIRAR O JOGO? OS CURSOS ESTÃO AQUI ⬇\nS...,UCNiU1wZxK6YN-KuJP7QMpBQ,2025-05-06 15:01:35,22,"['React Feminino', 'Auto ajuda', 'Superação', ...",...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Society'],NaN,0,0,0,NaN,luta silenciosa do homens #redpill #masculinid...
189,659,NaN,NaN,t8G3O1khwLk,"Cuidado com quem vai escolher... Uma destrói, ...",#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-17 15:01:22,22,[],...,0,NaN,NaN,[],NaN,0,0,0,NaN,cuidado com quem vai escolher uma destrói outr...
161,629,NaN,NaN,fB9-IWf4aQ4,Quando se tem uma grande MULHER te incentivand...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-19 15:01:07,22,[],...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0,0,0,NaN,quando se tem uma grande mulher te incentivand...


In [4]:
# Removendo todas as linhas onde o texto não é string
unified_df = unified_df[unified_df["text"].apply(lambda x: isinstance(x, str))]

# Resetando o índice para organizar o DataFrame após a remoção
unified_df.reset_index(drop=True, inplace=True)

print(f"Total de linhas: {len(unified_df)}")

Total de linhas: 447


In [5]:
import re
from collections import Counter
import pandas as pd

# Funções para capturar os padrões
def encontrar_emojis(text):
    # Regex para emojis
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF"
                               "\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF"
                               "\U00002700-\U000027BF\U0001F900-\U0001F9FF"
                               "\U0001FA70-\U0001FAFF\U00002600-\U000026FF"
                               "\U00002500-\U00002BEF\U0001F004\U0001F0CF"
                               "\U0001F018-\U0001F270\U0001F985-\U0001F9C0]+", 
                               flags=re.UNICODE)
    return emoji_pattern.findall(text)

def encontrar_mencoes(text):
    # Regex para menções
    return re.findall(r"@\w+", text)

def encontrar_urls(text):
    # Regex para URLs
    return re.findall(r"http[s]?://\S+", text)

def encontrar_hashtags(text):
    # Regex para hashtags
    return re.findall(r"#\w+", text)

# Aplicando as funções e coletando os resultados
emojis = []
mencoes = []
urls = []
hashtags = []

# Armazenando as relações de menções, emojis e hashtags
mencoes_relacoes = []
emojis_relacoes = []
hashtags_relacoes = []

# Iterando sobre o DataFrame
for index, row in unified_df.iterrows():
    texto = row["text"]
    
    # Verifica se é string antes de aplicar as funções
    if isinstance(texto, str):
        # Coleta emojis, menções, URLs e hashtags
        emojis_encontrados = encontrar_emojis(texto)
        mencao_encontradas = encontrar_mencoes(texto)
        urls.extend(encontrar_urls(texto))
        hashtags_encontradas = encontrar_hashtags(texto)
        
        # Adiciona os encontrados às listas
        emojis.extend(emojis_encontrados)
        mencoes.extend(mencao_encontradas)
        hashtags.extend(hashtags_encontradas)
        
        # Armazenando as relações "quem mencionou quem"
        autor = row.get("author", f"author_{index}")  # Se não tiver coluna 'author', usa índice
        
        # Quem mencionou author
        for mencao in mencao_encontradas:
            mencoes_relacoes.append({"author": autor, "mencionou": mencao})
        
        # Quem usou quais emojis
        for emoji in emojis_encontrados:
            emojis_relacoes.append({"author": autor, "emoji": emoji})
        
        # Quem usou quais hashtags
        for hashtag in hashtags_encontradas:
            hashtags_relacoes.append({"author": autor, "hashtag": hashtag})

# Contando os mais comuns
emojis_comuns = Counter(emojis).most_common(10)
mencoes_comuns = Counter([m.lower() for m in mencoes]).most_common(10)  # Ignora maiúsculas/minúsculas
urls_comuns = Counter(urls).most_common(10)
hashtags_comuns = Counter([h.lower() for h in hashtags]).most_common(10)  # Ignora maiúsculas/minúsculas

# Exibindo os resultados
print("Emojis mais comuns:", emojis_comuns)
print("Menções mais comuns:", mencoes_comuns)
print("URLs mais comuns:", urls_comuns)
print("Hashtags mais comuns:", hashtags_comuns)

Emojis mais comuns: [('🙏', 3)]
Menções mais comuns: [('@submundointelectual', 78), ('@redcastoficial', 78), ('@instituto__oliver_oficial', 39), ('@gmailcom', 23), ('@cortesdoconselho', 22), ('@sigma', 1), ('@rtao', 1)]
URLs mais comuns: []
Hashtags mais comuns: [('#redpillbrasil', 183), ('#rafaelaires', 176), ('#antiotario', 176), ('#memes', 112), ('#humor', 107), ('#comedia', 70), ('#relaciomento', 50), ('#amelhorredpillvemdelas', 32), ('#redpill', 29), ('#redpillfeminina', 24)]


In [6]:
# Criando DataFrames das relações
df_mencoes = pd.DataFrame(mencoes_relacoes)
df_emojis = pd.DataFrame(emojis_relacoes)
df_hashtags = pd.DataFrame(hashtags_relacoes)

In [7]:
# Exibindo as relações de menções
print("\nRelações de menções (author mencionou author):")
df_mencoes


Relações de menções (author mencionou author):


,author,mencionou
0,author_5,@instituto__oliver_oficial
1,author_5,@submundointelectual
2,author_5,@redcastoficial
3,author_5,@submundointelectual
4,author_5,@redcastoficial
...,...,...
237,author_446,@instituto__oliver_oficial
238,author_446,@submundointelectual
239,author_446,@redcastoficial
240,author_446,@submundointelectual


In [8]:
df_emojis

,author,emoji
0,author_331,🙏
1,author_382,🙏
2,author_412,🙏


In [9]:
try:
    # Analisando authorororororororor mais usou quais emojis
    emojis_por_autor = df_emojis.groupby(['author', 'emoji']).size().reset_index(name='quantidade')
    emojis_top_usuarios = emojis_por_autor.sort_values(by='quantidade', ascending=False)
    print("\nQuem mais usou quais emojis:")
    emojis_top_usuarios
except KeyError:
    print("No emojis is texts")


Quem mais usou quais emojis:


In [10]:
# Analisando author mais usou quais hashtags
hashtags_por_autor = df_hashtags.groupby(['author', 'hashtag']).size().reset_index(name='quantidade')
hashtags_top_usuarios = hashtags_por_autor.sort_values(by='quantidade', ascending=False)
print("\nQuem mais usou quais hashtags:")
hashtags_top_usuarios


Quem mais usou quais hashtags:


,author,hashtag,quantidade
1288,author_46,#memes,4
1370,author_64,#relacionamentos,3
1290,author_46,#musica,2
962,author_368,#memes,2
1082,author_393,#divórcio,2
...,...,...,...
496,author_253,#casamento,1
495,author_252,#redpillbrasil,1
494,author_252,#rafaelaires,1
493,author_252,#antiotario,1


In [11]:
try:
    # Analisando authorororororororor mais usou quais mencaos
    mencaos_por_autor = df_mencoes.groupby(['author', 'mencionou']).size().reset_index(name='quantidade')
    mencaos_top_usuarios = mencaos_por_autor.sort_values(by='quantidade', ascending=False)
    print("\nQuem mais usou quais mencaos:")
    mencaos_top_usuarios
except KeyError:
    print("No mencaos is texts")


Quem mais usou quais mencaos:


In [12]:
df_mencoes['mencionou'].value_counts()

mencionou
@submundointelectual          78
@redcastoficial               78
@instituto__oliver_oficial    39
@gmailcom                     23
@cortesdoconselho             22
@sigma                         1
@rtao                          1
Name: count, dtype: int64